In [ ]:
!pip install python-terrier pandas --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Importing all the necessary packages
import pickle
import bs4 as bs
import os
import pyterrier as pt
import pandas as pd
from threading import Thread
import ast
import json
import nltk
from nltk.tokenize import word_tokenize
from pyterrier.measures import *
import re
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
pd.set_option('display.max_colwidth', 150)
import csv

In [ ]:
import json
import pickle
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
index = pt.IndexFactory.of(f"/content/drive/MyDrive/TREC-DL-2019-Judged/TREC-DL-2019-judged-ours/data.properties")

In [ ]:
dataset = pt.get_dataset("irds:msmarco-document/trec-dl-2019/judged")
qrels = dataset.get_qrels()
topics = dataset.get_topics()

topics.to_json('/content/queries.json')
print("the length of topics is : " , len(topics))

the length of topics is :  43


In [ ]:
def get_evaluation(br, topics, b = None, k1 = None):
  res = pt.Experiment(
    [br],
    topics,
    qrels,
    verbose=True,
    round = 3,
    eval_metrics=["recall_1000", "ndcg_cut_10", "map"])

  print(b, k1)
  # print(res)

  print(f'recall@1000 = {res.loc[0, "recall_1000"]}')
  print(f'map = {res.loc[0, "map"]}')
  print(f'ndcg@10 = {res.loc[0, "ndcg_cut_10"]}')

  return [res.loc[0, "recall_1000"], res.loc[0, "map"], res.loc[0, "ndcg_cut_10"]]

In [ ]:
def letsDoIt1(topics):
  left, right = 0, 1
  precision = 1e-2

  while abs(right - left) > precision:
    mid = (left + right) / 2

    resLeft = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : mid - precision}), topics)
    resRight = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : mid + precision}), topics)

    if resLeft[2] >= resRight[2]:
      right = mid
    else:
      left = mid

  return (left + right) / 2

In [ ]:
def letsDoIt2(topics):
  left, right = 0, 2
  precision = 1e-2

  while abs(right - left) > precision:
    mid = (left + right) / 2

    resLeft = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : b, "bm25.k_1": mid - precision}), topics)
    resRight = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : b, "bm25.k_1": mid + precision}), topics)

    if resLeft[2] >= resRight[2]:
      right = mid
    else:
      left = mid

  return (left + right) / 2

In [ ]:
b = letsDoIt1(topics)
k1 = letsDoIt2(topics)
print(b, k1)

In [ ]:
def method2part1(topics, idx = 0):
  left, right = 0, 1
  precision = 1e-2

  while abs(right - left) > precision:
    mid = (left + right) / 2
    midLeft = (left + mid) / 2
    midRight = (mid + right) / 2

    resLeft = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : midLeft}), topics, midLeft, None)
    resRight = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : midRight}), topics, midRight, None)

    if resLeft[idx] >= resRight[idx]:
      right = mid
    else:
      left = mid

  return (left + right) / 2

In [ ]:
def method2part2(topics, idx = 0):
  left, right = 0, 2
  precision = 1e-2

  while abs(right - left) > precision:
    mid = (left + right) / 2

    midLeft = (left + mid) / 2
    midRight = (mid + right) / 2

    resLeft = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : b, "bm25.k_1": midLeft}), topics, b, midLeft)
    resRight = get_evaluation(pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : b, "bm25.k_1": midRight}), topics, b, midRight)

    if resLeft[idx] >= resRight[idx]:
      right = mid
    else:
      left = mid

  return (left + right) / 2

In [ ]:
b = method2part1(topics, 0)
k1 = method2part2(topics, 0)
print('\n\n\n', b, k1)
b = method2part1(topics, 1)
k1 = method2part2(topics, 1)
print('\n\n\n', b, k1)
b = method2part1(topics, 2)
k1 = method2part2(topics, 2)
print('\n\n\n', b, k1)

In [ ]:
res = pt.Experiment(
      [pt.BatchRetrieve(index, wmodel="BM25")],
      topics,
      qrels,
      verbose=True,
      round = 3,
      eval_metrics=["recall_1000", "ndcg_cut_10", "map"])
print(res)
bo1 = pt.rewrite.Bo1QueryExpansion(index)
res = pt.Experiment(
      [pt.BatchRetrieve(index, wmodel="BM25")
      >> bo1 >>
       pt.BatchRetrieve(index, wmodel="BM25")],
      topics,
      qrels,
      verbose=True,
      round = 3,
      eval_metrics=["recall_1000", "ndcg_cut_10", "map"])
print(res)

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

       name  recall_1000  ndcg_cut_10    map
0  BR(BM25)        0.706        0.539  0.335


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

                                                                                                                                                    name  \
0  Compose(Compose(BR(BM25), QueryExpansion(/content/drive/MyDrive/TREC-DL-2019-Judged/TREC-DL-2019-judged-ours/data.properties,3,10,<org.terrier.que...   

   recall_1000  ndcg_cut_10    map  
0        0.756        0.568  0.385  


In [ ]:
param_list = [[0.79296875, 1.94140625], [0.66015625, 1.54296875], [0.94140625, 1.94921875]]

In [ ]:
def final_evaluation(curr_topics, ls, add = 1):

  final_topics = curr_topics.copy(deep = True)

  for it in range(len(curr_topics)):
    final_topics.loc[it, 'query'] = add * topics.loc[it, 'query'] + ' ' + final_topics.loc[it, 'query']

  maxRecall, maxMap, maxNDCG = 0, 0, 0

  for it in range(len(ls)):
    print(ls[it])
    res = pt.Experiment(
      [pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : ls[it][0], "bm25.k_1": ls[it][1]})],
      final_topics,
      qrels,
      verbose=True,
      round = 3,
      eval_metrics=["recall_1000", "ndcg_cut_10", "map"])
    maxRecall = max(maxRecall, res.loc[0, "recall_1000"])
    maxMap = max(maxMap, res.loc[0, "map"])
    maxNDCG = max(maxNDCG, res.loc[0, "ndcg_cut_10"])

  print(f'recall@1000 = {res.loc[0, "recall_1000"]}')
  print(f'map = {res.loc[0, "map"]}')
  print(f'ndcg@10 = {res.loc[0, "ndcg_cut_10"]}')

  return [maxRecall, maxMap, maxNDCG]

In [ ]:
def final_evaluation2(curr_topics, ls, add = 1):

  bo1 = pt.rewrite.Bo1QueryExpansion(index)

  final_topics = curr_topics.copy(deep = True)

  for it in range(len(curr_topics)):
    final_topics.loc[it, 'query'] = add * topics.loc[it, 'query'] + ' ' + final_topics.loc[it, 'query']

  maxRecall, maxMap, maxNDCG = 0, 0, 0

  for it in range(len(ls)):
    print(ls[it])
    res = pt.Experiment(
      [pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : ls[it][0], "bm25.k_1": ls[it][1]})
      >> bo1 >>
       pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : ls[it][0], "bm25.k_1": ls[it][1]})],
      final_topics,
      qrels,
      verbose=True,
      round = 3,
      eval_metrics=["recall_1000", "ndcg_cut_10", "map"])
    maxRecall = max(maxRecall, res.loc[0, "recall_1000"])
    maxMap = max(maxMap, res.loc[0, "map"])
    maxNDCG = max(maxNDCG, res.loc[0, "ndcg_cut_10"])

  print(f'recall@1000 = {res.loc[0, "recall_1000"]}')
  print(f'map = {res.loc[0, "map"]}')
  print(f'ndcg@10 = {res.loc[0, "ndcg_cut_10"]}')

  return [maxRecall, maxMap, maxNDCG]

In [ ]:
print(final_evaluation(topics, param_list, 0))
print(final_evaluation2(topics, param_list, 0))

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.723
map = 0.355
ndcg@10 = 0.551
[0.753, 0.387, 0.569]


In [ ]:
folder_best = '/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/bestRecallMapNdcg'

In [ ]:
from itertools import combinations

def comb(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)
  for x in all_comb:
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    with open(f'{folder_best}/best_ndcg.csv', 'a') as f:
      csvwriter = csv.writer(f)
      for it in range(2, 3):
        res = [x] + final_evaluation2(df_comb, param_list, it) + [it]
        csvwriter.writerow(res)
        print(res)

In [ ]:
from itertools import combinations

def comb_bard(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)
  for x in all_comb:
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    with open(f'{folder_bard}/res_all_combined.csv', 'a') as f:
      csvwriter = csv.writer(f)
      for it in range(2, 3):
        res = [x] + final_evaluation2(df_comb, param_list, it) + [it]
        csvwriter.writerow(res)
        print(res)

In [ ]:
# from itertools import combinations

# def comb_gemini(ls, num_of_permutations):
#   all_comb = combinations(ls.keys(), num_of_permutations)
#   for x in all_comb:
#     df_comb = topics.copy(deep = True)
#     for k in x:
#       for it in range(len(df_comb)):
#         df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

#     with open(f'{folder_gemini}/res2.csv', 'a') as f:
#       csvwriter = csv.writer(f)
#       final_evaluation_value = [x] + final_evaluation2(df_comb,param_list, 1) + [1]
#       csvwriter.writerow(final_evaluation_value)
#       print(final_evaluation_value)
#       final_evaluation_value = [x] + final_evaluation2(df_comb,param_list, 2) + [2]
#       csvwriter.writerow(final_evaluation_value)
#       print(final_evaluation_value)


In [ ]:
from itertools import combinations

def comb_gemini(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)
  for x in all_comb:
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    with open(f'{folder_gemini}/res_all_combined.csv', 'a') as f:
      csvwriter = csv.writer(f)
      for it in range(1, 3):
        res = [x] + final_evaluation2(df_comb, param_list, it) + [it]
        csvwriter.writerow(res)
        print(res)

In [ ]:
# GPT territory starts from here
folder_gpt = '/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/GPT_Input_Queries'
# gpt_list = ['df_document_gpt', 'df_entities_gpt', 'df_entities_cot_gpt']
gpt_df = {}
gpt_list = ['df_document_gpt', 'df_entities_gpt', 'df_entities_cot_gpt', 'df_essay_gpt', 'df_facts_gpt', 'df_keywords_gpt', 'df_keywords_cot_gpt', 'df_keywords_diverse_gpt', 'df_news_gpt', 'df_summary_gpt', 'df_expanded_queries_gpt', 'df_web_page_gpt', 'df_domain_terms_gpt', 'df_closely_related_gpt', 'df_alt_sentences_gpt']


In [ ]:
for df_name in gpt_list:
  gpt_df[df_name] = pd.read_csv(f'{folder_gpt}/DataFrames/{df_name}.csv')
  gpt_df[df_name]['qid'] = gpt_df[df_name]['qid'].apply(str)

In [ ]:
csv_data = []

for df_name in gpt_list:
  for it in range(1, 6):
    res = final_evaluation(gpt_df[df_name], param_list, it)
    csv_data.append([df_name] + res + [it])

print(csv_data)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.742
map = 0.364
ndcg@10 = 0.565
[['df_alt_sentences_gpt', 0.769, 0.38, 0.593, 1], ['df_alt_sentences_gpt', 0.766, 0.383, 0.595, 2], ['df_alt_sentences_gpt', 0.764, 0.384, 0.596, 3], ['df_alt_sentences_gpt', 0.752, 0.379, 0.593, 4], ['df_alt_sentences_gpt', 0.746, 0.373, 0.587, 5]]


In [ ]:
with open(f'{folder_gpt}/res.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

In [ ]:
ls = {
    'df_document_gpt': gpt_df['df_document_gpt'],
    'df_news_gpt': gpt_df['df_news_gpt'],
    'df_facts_gpt': gpt_df['df_facts_gpt'],
    'df_alt_sentences_gpt' : gpt_df['df_alt_sentences_gpt']
}

for it in range(2,len(ls)+1):
  comb(ls, it)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

recall@1000 = 0.84
map = 0.476
ndcg@10 = 0.616
[('df_document_gpt', 'df_facts_gpt', 'df_alt_sentences_gpt'), 0.84, 0.476, 0.616, 3]
[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

KeyboardInterrupt: 

In [ ]:
# BARD territory starts from here
folder_bard = '/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/BARD_Input_Queries'
bard_list = ['df_document_bard', 'df_entities_bard', 'df_entities_cot_bard', 'df_essay_bard', 'df_facts_bard', 'df_keywords_bard', 'df_keywords_cot_bard', 'df_news_bard', 'df_summary_bard', 'df_alt_sentences_bard', 'df_web_page_bard', 'df_expanded_queries_bard']
bard_df = {}

In [ ]:
for df_name in bard_list:
  bard_df[df_name] = pd.read_csv(f'{folder_bard}/DataFrames/{df_name}.csv')
  bard_df[df_name]['qid'] = bard_df[df_name]['qid'].apply(str)

In [ ]:
print(bard_df)
#print(final_evaluation(bard_df['df_document_bard'], param_list, 1))

In [ ]:
csv_data_bard = []

for df_name in bard_list:
  for it in range(1,6):
    res = final_evaluation(bard_df[df_name], param_list, it)
    csv_data_bard.append([df_name] + res + [it])



[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.681
map = 0.316
ndcg@10 = 0.531
[]


In [ ]:
print(csv_data_bard)

with open(f'{folder_bard}/res2.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data_bard:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

[['df_web_page_bard', 0.749, 0.38, 0.568, 1], ['df_web_page_bard', 0.756, 0.387, 0.582, 2], ['df_web_page_bard', 0.754, 0.394, 0.587, 3], ['df_web_page_bard', 0.75, 0.392, 0.59, 4], ['df_web_page_bard', 0.745, 0.389, 0.591, 5], ['df_expanded_queries_bard', 0.742, 0.381, 0.583, 1], ['df_expanded_queries_bard', 0.731, 0.375, 0.578, 2], ['df_expanded_queries_bard', 0.725, 0.365, 0.574, 3], ['df_expanded_queries_bard', 0.714, 0.353, 0.564, 4], ['df_expanded_queries_bard', 0.699, 0.343, 0.548, 5]]


In [ ]:
csv_data_bard = []

for df_name in bard_list:
  for it in range(2, 3):
    res = final_evaluation2(bard_df[df_name], param_list, it)
    csv_data_bard.append([df_name] + res + [it])

print(csv_data_bard)

with open(f'{folder_bard}/res2.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data_bard:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.757
map = 0.396
ndcg@10 = 0.596
[['df_web_page_bard', 0.797, 0.432, 0.595, 2], ['df_expanded_queries_bard', 0.765, 0.398, 0.597, 2]]


In [ ]:
ls = {
    'df_document_bard': bard_df['df_document_bard'],
    'df_entities_cot_bard': bard_df['df_entities_cot_bard'],
    'df_essay_bard': bard_df['df_essay_bard'],
    'df_facts_bard' : bard_df['df_facts_bard']
}

for it in range(len(bard_df),len(bard_df)+1):
  comb_bard(bard_df, it)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.825
map = 0.483
ndcg@10 = 0.645
[('df_document_bard', 'df_entities_bard', 'df_entities_cot_bard', 'df_essay_bard', 'df_facts_bard', 'df_keywords_bard', 'df_keywords_cot_bard', 'df_news_bard', 'df_summary_bard', 'df_alt_sentences_bard', 'df_web_page_bard', 'df_expanded_queries_bard'), 0.83, 0.483, 0.645, 2]


In [ ]:
with open(f'{folder_bard}/res2.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data_bard:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

In [ ]:
# GEMINI territory starts from here
folder_gemini = '/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/GEMINI_Input_Queries'
gemini_df = {}
gemini_list = ['df_document_gemini', 'df_entities_gemini', 'df_entities_cot_gemini', 'df_essay_gemini', 'df_facts_gemini', 'df_keywords_gemini', 'df_keywords_cot_gemini', 'df_news_gemini', 'df_summary_gemini', 'df_expanded_queries_gemini', 'df_web_page_gemini', 'df_alt_sentences_gemini']

In [ ]:
for df_name in gemini_list:
  gemini_df[df_name] = pd.read_csv(f'{folder_gemini}/DataFrames/{df_name}.csv')
  gemini_df[df_name]['qid'] = gemini_df[df_name]['qid'].apply(str)

In [ ]:
print(gemini_df)

In [ ]:
csv_data_gemini = []

for df_name in gemini_list:
  for it in range(1, 3):
    res = final_evaluation2(gemini_df[df_name], param_list, it)
    csv_data_gemini.append([df_name] + res + [it])

print(csv_data_gemini)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

KeyboardInterrupt: 

In [ ]:
ls = {
    'df_facts_gemini': gemini_df['df_facts_gemini'],
    'df_document_gemini': gemini_df['df_document_gemini'],
    'df_alt_sentences_gemini': gemini_df['df_alt_sentences_gemini'],
    'df_web_page_gemini' : gemini_df['df_web_page_gemini']
}

for it in range(len(gemini_df), len(gemini_df)+1):
  comb_gemini(gemini_df, it)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.856
map = 0.504
ndcg@10 = 0.677
[('df_document_gemini', 'df_entities_gemini', 'df_entities_cot_gemini', 'df_essay_gemini', 'df_facts_gemini', 'df_keywords_gemini', 'df_keywords_cot_gemini', 'df_news_gemini', 'df_summary_gemini', 'df_expanded_queries_gemini', 'df_web_page_gemini', 'df_alt_sentences_gemini'), 0.856, 0.504, 0.677, 2]


In [ ]:
with open(f'{folder_gemini}/res.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data_gemini:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

In [ ]:
# b = method2part1(bard_df[bard_list[1]], 0)
# k1 = method2part2(bard_df[bard_list[1]], 0)
# print('\n\n\n', b, k1)
# b = method2part1(bard_df[bard_list[1]], 1)
# k1 = method2part2(bard_df[bard_list[1]], 1)
# print('\n\n\n', b, k1)
b = method2part1(bard_df[bard_list[1]], 2)
k1 = method2part2(bard_df[bard_list[1]], 2)
print('\n\n\n', b, k1)

In [ ]:
# dataset = pt.get_dataset('irds:msmarco-document')
# # Index msmarco-document
# indexer = pt.IterDictIndexer('/content/drive/MyDrive/Index/msmarco-document', meta={'docno': 20, 'title': 200}, meta_reverse=['docno', 'title'])
# index_ref = indexer.index(dataset.get_corpus_iter(), fields=['url', 'title'], meta=['docno', 'title'])
index = pt.IndexFactory.of(f"/content/drive/MyDrive/Index/msmarco-document/data.properties")

12:37:18.842 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored


In [ ]:
bm25 = (pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : 0.66015625, "bm25.k_1": 1.54296875}) % 5) >> pt.text.get_text(index, 'title')
res = bm25.transform(topics)
print(res)

In [ ]:
query_res = {}

for index, row in res.iterrows():
  if row['query'] in query_res.keys():
    query_res[row['query']].append(row['title'])
  else:
    query_res[row['query']] = [row['title']]

query_res_json = []

for key, value in query_res.items():
  query_res_json.append({'query': key})
  query_res_json[-1]['retrived_documents_title'] = value

In [ ]:
with open("/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/query_res.json", "w") as outfile:
    json.dump(query_res_json, outfile)

In [ ]:
folder_gpt4 = '/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/GPT4_Input_Queries'
gpt4_df = {}
#gpt_list = ['df_document_gpt', 'df_entities_gpt', 'df_entities_cot_gpt', 'df_essay_gpt', 'df_facts_gpt', 'df_keywords_gpt', 'df_keywords_cot_gpt', 'df_keywords_diverse_gpt', 'df_news_gpt', 'df_summary_gpt', 'df_expanded_queries_gpt', 'df_web_page_gpt', 'df_domain_terms_gpt', 'df_closely_related_gpt', 'df_alt_sentences_gpt']
gpt4_list = ['df_document_gpt4', 'df_news_gpt4', 'df_essay_gpt4', 'df_summary_gpt4', 'df_facts_gpt4', 'df_keywords_gpt4', 'df_web_page_gpt4']
need_improv = ['df_entities_gpt4', 'df_entities_cot_gpt4', 'df_keywords_cot_gpt4', 'df_expanded_queries_gpt4', 'df_alt_sentences_gpt4']

gpt4_list += need_improv
# gpt4_list =  ['df_alt_sentences_gpt4']

In [ ]:
for df_name in gpt4_list:
  gpt4_df[df_name] = pd.read_csv(f'{folder_gpt4}/DataFrames/{df_name}.csv')
  gpt4_df[df_name]['qid'] = gpt4_df[df_name]['qid'].apply(str)

In [ ]:
csv_data_gpt4 = []

for df_name in gpt4_list:
  for it in range(1,6):
    res = final_evaluation(gpt4_df[df_name], param_list, it)
    csv_data_gpt4.append([df_name] + res + [it])


print(csv_data_gpt4)

with open(f'{folder_gpt4}/res2.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data_gpt4:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.704
map = 0.328
ndcg@10 = 0.529
[['df_alt_sentences_gpt4', 0.737, 0.37, 0.563, 1], ['df_alt_sentences_gpt4', 0.73, 0.367, 0.567, 2], ['df_alt_sentences_gpt4', 0.73, 0.362, 0.569, 3], ['df_alt_sentences_gpt4', 0.727, 0.359, 0.566, 4], ['df_alt_sentences_gpt4', 0.726, 0.353, 0.559, 5]]


In [ ]:
# csv_data_gpt4 = []

# for df_name in gpt4_list:
#   res = final_evaluation2(gpt4_df[df_name], param_list, 2)

print(res)
csv_data_gpt4.append([df_name] + res)

print(csv_data_gpt4)

with open(f'{folder_gpt4}/res2.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data_gpt4:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

[0.762, 0.4, 0.592]
[['df_alt_sentences_gpt4', 0.762, 0.4, 0.592]]


In [ ]:
print(csv_data)

[['df_expanded_queries_gpt4', 0.826, 0.443, 0.598]]


In [ ]:
with open(f'{folder_gpt4}/res.csv', 'a') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow('\n')
  for data in csv_data:
    csvwriter.writerow(data)
  csvwriter.writerow('\n')

In [ ]:
ls = {
    'df_document_gpt4': gpt4_df['df_document_gpt4'],
    'df_news_gpt4': gpt4_df['df_news_gpt4'],
    'df_essay_gpt4': gpt4_df['df_essay_gpt4'],
    'df_summary_gpt4' : gpt4_df['df_summary_gpt4']
}

for it in range(2,len(ls)+1):
  comb(ls, it)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

In [ ]:
res = final_evaluation(df, param_list)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.717
map = 0.353
ndcg@10 = 0.563


In [ ]:
rag_document_bard = topics.copy(deep = True)

with open('/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/RAG_BARD_Input_Queries/rag_document_bard.json', 'r') as jsonFile:
  ls = json.load(jsonFile)

for it in range(len(rag_document_bard)):
  rag_document_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', ls[it]['document'])

In [ ]:
res = final_evaluation(rag_document_bard, param_list)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.76
map = 0.39
ndcg@10 = 0.579


In [ ]:
rag_answer_gpt4 = topics.copy(deep = True)

with open('/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/RAG_GPT_Input_Queries/rag_gpt4.json', 'r') as jsonFile:
  ls = json.load(jsonFile)

for it in range(len(rag_answer_gpt4)):
  rag_answer_gpt4.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', ls[it]['answer'])

In [ ]:
res = final_evaluation(rag_answer_gpt, param_list)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.769
map = 0.389
ndcg@10 = 0.578


In [ ]:
res = final_evaluation(rag_answer_gpt4, param_list)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.791
map = 0.408
ndcg@10 = 0.614


In [ ]:
folder_rag = '/content/drive/MyDrive/Trec-DL-2019-Judged-regenerated/IR_indexes/RAG_Input_Regenerated'
rag_df = {}
rag_list = ['df_rag_gemini']

In [ ]:
for df_name in rag_list:
  rag_df[df_name] = pd.read_csv(f'{folder_rag}/DataFrames/{df_name}.csv')
  rag_df[df_name]['qid'] = rag_df[df_name]['qid'].apply(str)

csv_data = []

for df_name in rag_list:
  for it in range(1, 6):
    res = final_evaluation2(rag_df[df_name], param_list, it)
    csv_data.append([df_name] + res + [it])

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


...

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.809
map = 0.439
ndcg@10 = 0.601


In [ ]:
ls = {
    'df_facts_gemini': gemini_df['df_facts_gemini'],
    'df_document_gemini': gemini_df['df_document_gemini'],
    'df_alt_sentences_gemini': gemini_df['df_alt_sentences_gemini'],
    'df_web_page_gemini' : gemini_df['df_web_page_gemini'],
    'df_document_gpt4': gpt4_df['df_document_gpt4'],
    'df_summary_gpt4': gpt4_df['df_summary_gpt4'],
    'df_document_gpt': gpt_df['df_document_gpt'],
    'df_alt_sentences_gpt': gpt_df['df_alt_sentences_gpt'],
    'df_document_bard': bard_df['df_document_bard'],
    'df_essay_bard': bard_df['df_essay_bard']
}

for it in range(len(ls),len(ls)+1):
  comb(ls, it)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.856
map = 0.508
ndcg@10 = 0.676
[('df_facts_gemini', 'df_document_gemini', 'df_alt_sentences_gemini', 'df_web_page_gemini', 'df_document_gpt4', 'df_summary_gpt4', 'df_document_gpt', 'df_alt_sentences_gpt', 'df_document_bard', 'df_essay_bard'), 0.856, 0.508, 0.676, 2]


In [ ]:
for key in ls:
  print(key)

df_facts_gemini
df_document_gemini
df_alt_sentences_gemini
df_web_page_gemini
df_document_gpt4
df_summary_gpt4
df_document_gpt
df_alt_sentences_gpt
df_document_bard
df_essay_bard


In [ ]:
ls = {
    'df_facts_gemini': gemini_df['df_facts_gemini'],
    'df_document_gemini': gemini_df['df_document_gemini'],
    'df_alt_sentences_gemini': gemini_df['df_alt_sentences_gemini'],
    'df_web_page_gemini' : gemini_df['df_web_page_gemini'],
    'df_document_gpt4': gpt4_df['df_document_gpt4'],
    'df_news_gpt4': gpt4_df['df_news_gpt4'],
    'df_document_gpt': gpt_df['df_document_gpt'],
    'df_alt_sentences_gpt': gpt_df['df_alt_sentences_gpt'],
    'df_facts_bard': bard_df['df_facts_bard'],
    'df_document_bard': bard_df['df_document_bard'],
    'df_entities_cot_bard': bard_df['df_entities_cot_bard'],
    'df_essay_bard' : bard_df['df_essay_bard'],
}

for it in range(len(ls),len(ls)+1):
  comb(ls, it)


[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.857
map = 0.516
ndcg@10 = 0.695
[('df_facts_gemini', 'df_document_gemini', 'df_alt_sentences_gemini', 'df_web_page_gemini', 'df_document_gpt4', 'df_news_gpt4', 'df_document_gpt', 'df_alt_sentences_gpt', 'df_facts_bard', 'df_document_bard', 'df_entities_cot_bard', 'df_essay_bard'), 0.857, 0.516, 0.695, 2]


In [ ]:
ls = {
    'df_facts_gemini': gemini_df['df_facts_gemini'],
    'df_alt_sentences_gemini': gemini_df['df_alt_sentences_gemini'],
    'df_document_gpt4': gpt4_df['df_document_gpt4'],
    'df_news_gpt4': gpt4_df['df_news_gpt4'],
    'df_document_gpt': gpt_df['df_document_gpt'],
    'df_alt_sentences_gpt': gpt_df['df_alt_sentences_gpt'],
    'df_facts_bard': bard_df['df_facts_bard'],
    'df_document_bard': bard_df['df_document_bard'],
    'df_entities_cot_bard': bard_df['df_entities_cot_bard'],
    'df_essay_bard' : bard_df['df_essay_bard'],
}

for it in range(len(ls),len(ls)+1):
  comb(ls, it)

# for key in ls:
#   print(key)

[0.79296875, 1.94140625]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.66015625, 1.54296875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

[0.94140625, 1.94921875]


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.858
map = 0.515
ndcg@10 = 0.691
[('df_facts_gemini', 'df_alt_sentences_gemini', 'df_document_gpt4', 'df_news_gpt4', 'df_document_gpt', 'df_alt_sentences_gpt', 'df_facts_bard', 'df_document_bard', 'df_entities_cot_bard', 'df_essay_bard'), 0.858, 0.515, 0.691, 2]
